In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import math as math

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [6]:
# NEURAL NETWORK COM 1 HIDDEN LAYER COM REGULARIZATION

num_nodes = 1024
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hidden_layer_weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
  hidden_layer_biases = tf.Variable(tf.zeros([num_nodes])) 
  output_weights = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # Hidden layer first
  hidden_layer_logits = tf.add(tf.matmul(tf_train_dataset, hidden_layer_weights), hidden_layer_biases)
  hidden_layer_relu = tf.nn.relu(hidden_layer_logits)

  # Then, output Layer
  logits = tf.add(tf.matmul(hidden_layer_relu, output_weights), output_biases)
  intermediate_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
  regularizers = tf.nn.l2_loss(hidden_layer_weights) + tf.nn.l2_loss(output_weights)
  loss = tf.reduce_mean(intermediate_loss + beta * regularizers)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training
  train_prediction = tf.nn.softmax(logits)

  # Predictions for validation
  # Hidden layer first, and then output Layer
  hidden_layer_logits = tf.add(tf.matmul(tf_valid_dataset, hidden_layer_weights), hidden_layer_biases)
  hidden_layer_relu = tf.nn.relu(hidden_layer_logits)
  logits = tf.matmul(hidden_layer_relu, output_weights) + output_biases
  
  valid_prediction = tf.nn.softmax(logits)

  # Predictions for test
  hidden_layer_logits = tf.add(tf.matmul(tf_test_dataset, hidden_layer_weights), hidden_layer_biases)
  hidden_layer_relu = tf.nn.relu(hidden_layer_logits)
  logits = tf.matmul(hidden_layer_relu, output_weights) + output_biases
  
  test_prediction = tf.nn.softmax(logits)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3545.674316
Minibatch accuracy: 3.1%
Validation accuracy: 31.1%
Minibatch loss at step 500: 21.562208
Minibatch accuracy: 74.2%
Validation accuracy: 83.9%
Minibatch loss at step 1000: 0.902181
Minibatch accuracy: 84.4%
Validation accuracy: 84.3%
Minibatch loss at step 1500: 0.801053
Minibatch accuracy: 82.8%
Validation accuracy: 84.2%
Minibatch loss at step 2000: 0.718346
Minibatch accuracy: 82.8%
Validation accuracy: 83.9%
Minibatch loss at step 2500: 0.808015
Minibatch accuracy: 81.2%
Validation accuracy: 83.9%
Minibatch loss at step 3000: 0.730447
Minibatch accuracy: 84.4%
Validation accuracy: 84.3%
Test accuracy: 90.7%


In [6]:
# DEMO DE OVERFITTING COM NEURAL NETWORK COM 1 HIDDEN LAYER COM REGULARIZATION

num_steps = 3001

new_train_dataset = train_dataset[:500, :]
new_train_labels = train_labels[:500]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (new_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = new_train_dataset[offset:(offset + batch_size), :]
    batch_labels = new_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3458.059082
Minibatch accuracy: 12.5%
Validation accuracy: 31.9%
Minibatch loss at step 500: 21.032618
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 1000: 0.490188
Minibatch accuracy: 100.0%
Validation accuracy: 78.9%
Minibatch loss at step 1500: 0.319088
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 2000: 0.294402
Minibatch accuracy: 100.0%
Validation accuracy: 79.1%
Minibatch loss at step 2500: 0.287087
Minibatch accuracy: 100.0%
Validation accuracy: 79.2%
Minibatch loss at step 3000: 0.278630
Minibatch accuracy: 100.0%
Validation accuracy: 79.2%
Test accuracy: 85.3%


In [7]:
# NEURAL NETWORK COM 1 HIDDEN LAYER COM DROPOUT

num_nodes = 1024
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hidden_layer_weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
  hidden_layer_biases = tf.Variable(tf.zeros([num_nodes])) 
  output_weights = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # Hidden layer first
  hidden_layer_logits = tf.matmul(tf_train_dataset, hidden_layer_weights) + hidden_layer_biases
  hidden_layer_relu = tf.nn.relu(hidden_layer_logits)
  # Dropout on hidden layer: RELU layer
  keep_prob = tf.placeholder("float")
  relu_layer_dropout = tf.nn.dropout(hidden_layer_relu, keep_prob)
  # Then, output Layer
  logits = tf.add(tf.matmul(relu_layer_dropout, output_weights), output_biases)
  intermediate_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
  regularizers = tf.nn.l2_loss(hidden_layer_weights) + tf.nn.l2_loss(output_weights)
  loss = tf.reduce_mean(intermediate_loss + beta * regularizers)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training
  train_prediction = tf.nn.softmax(logits)

  # Predictions for validation
  # Hidden layer first, and then output Layer
  hidden_layer_logits = tf.add(tf.matmul(tf_valid_dataset, hidden_layer_weights), hidden_layer_biases)
  hidden_layer_relu = tf.nn.relu(hidden_layer_logits)
  logits = tf.matmul(hidden_layer_relu, output_weights) + output_biases
  
  valid_prediction = tf.nn.softmax(logits)

  # Predictions for test
  hidden_layer_logits = tf.add(tf.matmul(tf_test_dataset, hidden_layer_weights), hidden_layer_biases)
  hidden_layer_relu = tf.nn.relu(hidden_layer_logits)
  logits = tf.matmul(hidden_layer_relu, output_weights) + output_biases
  
  test_prediction = tf.nn.softmax(logits)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3673.822754
Minibatch accuracy: 6.2%
Validation accuracy: 35.0%
Minibatch loss at step 500: 21.656736
Minibatch accuracy: 70.3%
Validation accuracy: 84.2%
Minibatch loss at step 1000: 0.952273
Minibatch accuracy: 84.4%
Validation accuracy: 83.8%
Minibatch loss at step 1500: 0.939954
Minibatch accuracy: 78.1%
Validation accuracy: 83.6%
Minibatch loss at step 2000: 0.794570
Minibatch accuracy: 81.2%
Validation accuracy: 83.7%
Minibatch loss at step 2500: 0.880424
Minibatch accuracy: 79.7%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 0.781622
Minibatch accuracy: 84.4%
Validation accuracy: 84.0%
Test accuracy: 90.3%


In [36]:
# DEMO DE OVERFITTING COM NEURAL NETWORK COM 1 HIDDEN LAYER COM REGULARIZATION

num_steps = 3001

new_train_dataset = train_dataset[:500, :]
new_train_labels = train_labels[:500]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (new_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = new_train_dataset[offset:(offset + batch_size), :]
    batch_labels = new_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3579.530273
Minibatch accuracy: 15.6%
Validation accuracy: 42.3%
Minibatch loss at step 500: 21.114262
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Minibatch loss at step 1000: 0.548569
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Minibatch loss at step 1500: 0.366212
Minibatch accuracy: 100.0%
Validation accuracy: 79.2%
Minibatch loss at step 2000: 0.332026
Minibatch accuracy: 100.0%
Validation accuracy: 79.2%
Minibatch loss at step 2500: 0.323655
Minibatch accuracy: 99.2%
Validation accuracy: 79.5%
Minibatch loss at step 3000: 0.311116
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Test accuracy: 85.5%


In [ ]:
# NEURAL NETWORK COM 1 HIDDEN LAYER COM DROPOUT

batch_size = 128
beta = 0.001

# Number of nodes decrease by 50% with each hidden layer that is deeper in the neural net
hidden_nodes = [ image_size * image_size ] 
hidden_nodes += [ int(1024 * np.power(0.5, i)) for i in range(5)]
hidden_nodes = hidden_nodes + [num_labels]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  




  intermediate_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
  regularizers = tf.nn.l2_loss(hidden_layer_weights) + tf.nn.l2_loss(output_weights)
  loss = tf.reduce_mean(intermediate_loss + beta * regularizers)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training
  train_prediction = tf.nn.softmax(logits)

  # Predictions for validation
  # Hidden layer first, and then output Layer
  hidden_layer_logits = tf.add(tf.matmul(tf_valid_dataset, hidden_layer_weights), hidden_layer_biases)
  hidden_layer_relu = tf.nn.relu(hidden_layer_logits)
  logits = tf.matmul(hidden_layer_relu, output_weights) + output_biases
  
  valid_prediction = tf.nn.softmax(logits)

  # Predictions for test
  hidden_layer_logits = tf.add(tf.matmul(tf_test_dataset, hidden_layer_weights), hidden_layer_biases)
  hidden_layer_relu = tf.nn.relu(hidden_layer_logits)
  logits = tf.matmul(hidden_layer_relu, output_weights) + output_biases
  
  test_prediction = tf.nn.softmax(logits)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [7]:
hidden_nodes = [ image_size * image_size ] 
hidden_nodes += [ int(1024 * np.power(0.5, i)) for i in range(5)]
hidden_nodes = hidden_nodes + [num_labels]

# Variables and Training
hidden_layer_weights = []
hidden_layer_biases = []
hidden_layer = []
hidden_layer_dropout = []
keep_prob = tf.placeholder("float")
for i in range(5):
    hidden_layer_weights += [tf.Variable(tf.truncated_normal([hidden_nodes[i], hidden_nodes[i+1]], stddev=math.sqrt(2.0/hidden_nodes[i])))]
    hidden_layer_biases += [tf.Variable(tf.zeros([hidden_nodes[i+1]]))]
    if not i:
        logits = [tf.add(tf.matmul(tf_train_dataset, hidden_layer_weights[i]), hidden_layer_biases[i])]
    else:
        logits += [tf.add(tf.matmul(hidden_layer_dropout[i-1], hidden_layer_weights[i]), hidden_layer_biases[i])]
    hidden_layer += [tf.nn.relu(logits[i])]
    hidden_layer_dropout += [tf.nn.dropout(hidden_layer[i], keep_prob)]
    
logits += [tf.add(tf.matmul(hidden_layer_dropout[i], hidden_layer_weights[i+1]), hidden_layer_biases[i+1])]

ValueError: Tensor("Variable_2:0", shape=(784, 1024), dtype=float32_ref) must be from the same graph as Tensor("Placeholder:0", shape=(128, 784), dtype=float32).

In [8]:
hidden_nodes = [ image_size * image_size ] 
hidden_nodes += [ int(1024 * np.power(0.5, i)) for i in range(5)]
hidden_nodes = hidden_nodes + [num_labels]

# Variables and Training
keep_prob = tf.placeholder("float")
for i in range(5):
    hidden_layer_weights = tf.Variable(tf.truncated_normal([hidden_nodes[i], hidden_nodes[i+1]], stddev=math.sqrt(2.0/hidden_nodes[i])))
    hidden_layer_biases = tf.Variable(tf.zeros([hidden_nodes[i+1]]))
    if not i:
        logits = tf.add(tf.matmul(tf_train_dataset, hidden_layer_weights), hidden_layer_biases)
    else:
        logits = tf.add(tf.matmul(hidden_layer_dropout, hidden_layer_weights), hidden_layer_biases)
    hidden_layer = tf.nn.relu(logits)
    hidden_layer_dropout = tf.nn.dropout(hidden_layer, keep_prob)
    
logits = tf.add(tf.matmul(hidden_layer_dropout, hidden_layer_weights), hidden_layer_biases)

ValueError: Tensor("Variable_4:0", shape=(784, 1024), dtype=float32_ref) must be from the same graph as Tensor("Placeholder:0", shape=(128, 784), dtype=float32).

In [53]:
batch_size = 128
beta = 0.001

hidden_nodes_1 = 1024
hidden_nodes_2 = int(hidden_nodes_1 * 0.5)
hidden_nodes_3 = int(hidden_nodes_1 * np.power(0.5, 2))
hidden_nodes_4 = int(hidden_nodes_1 * np.power(0.5, 3))
hidden_nodes_5 = int(hidden_nodes_1 * np.power(0.5, 4))

graph = tf.Graph()
with graph.as_default():

    '''Input Data'''
    # For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    '''Variables'''
    # Hidden RELU layer 1
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_1], stddev=math.sqrt(2.0/(image_size*image_size))))
    biases_1 = tf.Variable(tf.zeros([hidden_nodes_1]))

    # Hidden RELU layer 2
    weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes_1, hidden_nodes_2], stddev=math.sqrt(2.0/hidden_nodes_1)))
    biases_2 = tf.Variable(tf.zeros([hidden_nodes_2]))
    
    # Hidden RELU layer 3
    weights_3 = tf.Variable(tf.truncated_normal([hidden_nodes_2, hidden_nodes_3], stddev=math.sqrt(2.0/hidden_nodes_2)))
    biases_3 = tf.Variable(tf.zeros([hidden_nodes_3]))
    
    # Hidden RELU layer 4
    weights_4 = tf.Variable(tf.truncated_normal([hidden_nodes_3, hidden_nodes_4], stddev=math.sqrt(2.0/hidden_nodes_3)))
    biases_4 = tf.Variable(tf.zeros([hidden_nodes_4]))
    
    # Hidden RELU layer 5
    weights_5 = tf.Variable(tf.truncated_normal([hidden_nodes_4, hidden_nodes_5], stddev=math.sqrt(2.0/hidden_nodes_4)))
    biases_5 = tf.Variable(tf.zeros([hidden_nodes_5]))
    
    # Output layer
    weights_6 = tf.Variable(tf.truncated_normal([hidden_nodes_5, num_labels], stddev=math.sqrt(2.0/hidden_nodes_5)))
    biases_6 = tf.Variable(tf.zeros([num_labels]))
    
    '''Training computation'''
    
    # Hidden RELU layer 1
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    hidden_layer_1 = tf.nn.relu(logits_1)
    # Dropout on hidden layer: RELU layer
    keep_prob = tf.placeholder("float")
    hidden_layer_1_dropout = tf.nn.dropout(hidden_layer_1, keep_prob)
    
    
    # Hidden RELU layer 2
    logits_2 = tf.matmul(hidden_layer_1_dropout, weights_2) + biases_2
    hidden_layer_2 = tf.nn.relu(logits_2)
    # Dropout on hidden layer: RELU layer
    hidden_layer_2_dropout = tf.nn.dropout(hidden_layer_2, keep_prob)
    
    # Hidden RELU layer 3
    logits_3 = tf.matmul(hidden_layer_2_dropout, weights_3) + biases_3
    hidden_layer_3 = tf.nn.relu(logits_3)
    # Dropout on hidden layer: RELU layer
    hidden_layer_3_dropout = tf.nn.dropout(hidden_layer_3, keep_prob)
    
    # Hidden RELU layer 4
    logits_4 = tf.matmul(hidden_layer_3_dropout, weights_4) + biases_4
    hidden_layer_4 = tf.nn.relu(logits_4)
    # Dropout on hidden layer: RELU layer

    hidden_layer_4_dropout = tf.nn.dropout(hidden_layer_4, keep_prob)
    
    # Hidden RELU layer 5
    logits_5 = tf.matmul(hidden_layer_4_dropout, weights_5) + biases_5
    hidden_layer_5 = tf.nn.relu(logits_5)
    # Dropout on hidden layer: RELU layer
    hidden_layer_5_dropout = tf.nn.dropout(hidden_layer_5, keep_prob)
    
    # Output layer
    logits_6 = tf.matmul(hidden_layer_5_dropout, weights_6) + biases_6 
    
    # Normal loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_6, labels=tf_train_labels))
    # Loss function with L2 Regularization with decaying learning rate beta=0.5
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + \
                   tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_4) + \
                   tf.nn.l2_loss(weights_5) + tf.nn.l2_loss(weights_6)
    loss = tf.reduce_mean(loss + beta * regularizers)

    '''Optimizer'''
    # Decaying learning rate
    global_step = tf.Variable(0)  # count the number of steps taken.
    start_learning_rate = 0.5
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training
    train_prediction = tf.nn.softmax(logits_6)
    
    # Predictions for validation 
    valid_logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    valid_relu_1 = tf.nn.relu(valid_logits_1)
    
    valid_logits_2 = tf.matmul(valid_relu_1, weights_2) + biases_2
    valid_relu_2 = tf.nn.relu(valid_logits_2)
    
    valid_logits_3 = tf.matmul(valid_relu_2, weights_3) + biases_3
    valid_relu_3 = tf.nn.relu(valid_logits_3)
    
    valid_logits_4 = tf.matmul(valid_relu_3, weights_4) + biases_4
    valid_relu_4 = tf.nn.relu(valid_logits_4)
    
    valid_logits_5 = tf.matmul(valid_relu_4, weights_5) + biases_5
    valid_relu_5 = tf.nn.relu(valid_logits_5)
    
    valid_logits_6 = tf.matmul(valid_relu_5, weights_6) + biases_6
    
    valid_prediction = tf.nn.softmax(valid_logits_6)
    
    # Predictions for test
    test_logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    test_relu_1 = tf.nn.relu(test_logits_1)
    
    test_logits_2 = tf.matmul(test_relu_1, weights_2) + biases_2
    test_relu_2 = tf.nn.relu(test_logits_2)
    
    test_logits_3 = tf.matmul(test_relu_2, weights_3) + biases_3
    test_relu_3 = tf.nn.relu(test_logits_3)
    
    test_logits_4 = tf.matmul(test_relu_3, weights_4) + biases_4
    test_relu_4 = tf.nn.relu(test_logits_4)
    
    test_logits_5 = tf.matmul(test_relu_4, weights_5) + biases_5
    test_relu_5 = tf.nn.relu(test_logits_5)
    
    test_logits_6 = tf.matmul(test_relu_5, weights_6) + biases_6
    
    test_prediction = tf.nn.softmax(test_logits_6)

In [54]:
num_steps = 15000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 4.799793720245361
Minibatch accuracy: 10.9
Validation accuracy: 13.6
Minibatch loss at step 500: 1.9860926866531372
Minibatch accuracy: 71.1
Validation accuracy: 81.7
Minibatch loss at step 1000: 1.4631098508834839
Minibatch accuracy: 82.8
Validation accuracy: 84.5
Minibatch loss at step 1500: 1.2226948738098145
Minibatch accuracy: 78.9
Validation accuracy: 84.8
Minibatch loss at step 2000: 0.9524246454238892
Minibatch accuracy: 82.8
Validation accuracy: 85.9
Minibatch loss at step 2500: 1.0873297452926636
Minibatch accuracy: 83.6
Validation accuracy: 85.8
Minibatch loss at step 3000: 0.8996708393096924
Minibatch accuracy: 83.6
Validation accuracy: 86.2
Minibatch loss at step 3500: 0.8864556550979614
Minibatch accuracy: 80.5
Validation accuracy: 85.5
Minibatch loss at step 4000: 0.8612145781517029
Minibatch accuracy: 85.2
Validation accuracy: 86.5
Minibatch loss at step 4500: